# Refactor module & rename: *svs_file_to_patches_tfrecord(...)*
## Patch Names as encoding of Class, location and label

<img src='./images/file_name_coding.png' />

In [1]:
patch_image_name_dict = {'case_id': 'd83cc7d1c94', 
                         'location_x': 100, 
                         'location_y': 340, 
                         'class_label': 'dermis', 
                         'file_type': '.jpg'}

In [2]:
import os
import sys

import numpy as np

def dict_to_patch_name(patch_image_name_dict):
    """ Usage:
    patch_name = dict_to_patch_name(patch_image_name_dict) 
    
    Args:
        patch_image_name_dict:  {'case_id': 'd83cc7d1c94', 
                                 'location_x': 100, 
                                 'location_y': 340, 
                                 'class_label': 'dermis', 
                                 'file_type': '.jpg' }
        
    Returns:
        patch_name:     file name (without directory path)
    """
    if file_ext[0] != '.':
        file_ext = '.' + file_ext
    patch_name = base_name + '_row_%i_%i'%(row[0], row[1])
    patch_name += '_col_%i_%i%s'%(col[0], col[1], file_ext)
    
    return patch_name


def dict_from_patch_name(patch_file_name):
    """ Usage:
    patch_image_name_dict = dict_from_patch_name(patch_file_name)
    
    Args:
        fname:          file name as created by get_patch_name()
    Returns:
        patch_image_name_dict:  {'case_id': 'd83cc7d1c94', 
                                 'location_x': 100, 
                                 'location_y': 340, 
                                 'class_label': 'dermis', 
                                 'file_type': '.jpg' }
    """
    patch_image_name_dict = {'case_id': 'd83cc7d1c94', 
                             'location_x': 100, 
                             'location_y': 340, 
                             'class_label': 'dermis', 
                             'file_type': '.jpg'}    
    
    return patch_image_name_dict
